# Keys & Addresses — Python Teaching Notebook

Use this notebook to demonstrate Bitcoin & Ethereum keys, addresses, and checksums. Each section has short, runnable examples you can live‑code while teaching.

**Note:** A few cells use external libraries. If running in a fresh env, run the install cell first.

In [15]:
# Optional: install dependencies if needed
# Uncomment the next lines if you don't already have these packages installed in your environment.
# %pip install ecdsa eth-utils eth-keys 'eth-hash[pycryptodome]'

## Table of Contents
1. [Base58 & Base58Check (pure‑Python)](#base58)
2. [secp256k1 keys, ECDSA, and WIF](#secp256k1)
3. [Bitcoin P2PKH (legacy `1…`) address](#p2pkh)
4. [Bitcoin P2SH (`3…`) from a redeem script](#p2sh)
5. [Bech32 (SegWit v0) P2WPKH `bc1…`](#bech32)
6. [Bech32m (v1+, e.g., Taproot)](#bech32m)
7. [Ethereum: Keccak‑256, address derivation, EIP‑55](#ethereum)
8. [EIP‑55 checksum (pure‑Python)](#eip55)
9. [SHA‑256 vs Keccak‑256 quick compare](#hashcompare)
10. [Checksum demo: error detection](#checksumdemo)


## 1) Base58 & Base58Check (pure‑Python) <a id='base58'></a>
Why Base58 avoids confusing characters; how Base58Check uses a 4‑byte double‑SHA‑256 checksum to catch typos.

In [16]:
import hashlib

ALPHABET = '123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz'

def b58encode(b: bytes) -> str:
    n_zeros = len(b) - len(b.lstrip(b'\x00'))
    num = int.from_bytes(b, 'big')
    s = ''
    while num:
        num, rem = divmod(num, 58)
        s = ALPHABET[rem] + s
    return '1' * n_zeros + s

def b58decode(s: str) -> bytes:
    num = 0
    for ch in s:
        num = num * 58 + ALPHABET.index(ch)
    full = num.to_bytes((num.bit_length() + 7) // 8, 'big') or b'\x00'
    n_zeros = len(s) - len(s.lstrip('1'))
    return b'\x00' * n_zeros + full

def base58check_encode(version_byte: bytes, payload: bytes) -> str:
    data = version_byte + payload
    checksum = hashlib.sha256(hashlib.sha256(data).digest()).digest()[:4]
    return b58encode(data + checksum)

def base58check_decode(s: str) -> tuple[bytes, bytes]:
    raw = b58decode(s)
    data, checksum = raw[:-4], raw[-4:]
    calc = hashlib.sha256(hashlib.sha256(data).digest()).digest()[:4]
    if calc != checksum:
        raise ValueError('Bad Base58Check checksum')
    return data[0:1], data[1:]

# Demo
v, payload = b'\x00', bytes.fromhex('00112233445566778899aabbccddeeff00112233')
addr = base58check_encode(v, payload)
print('Base58Check:', addr)
print('Decoded:', base58check_decode(addr))

Base58Check: 11MXTrefsj1ZS3Q5e9D6DxGzZKHWALyo9
Decoded: (b'\x00', b'\x00\x11"3DUfw\x88\x99\xaa\xbb\xcc\xdd\xee\xff\x00\x11"3')


## 2) secp256k1 keys, ECDSA sign/verify, and WIF <a id='secp256k1'></a>
Deterministic ECDSA (RFC 6979), compressed vs uncompressed pubkeys, WIF encoding.

In [17]:
from ecdsa import SigningKey, SECP256k1
import os, hashlib

# Private key & ECDSA keypair
priv_key_bytes = os.urandom(32)
sk = SigningKey.from_string(priv_key_bytes, curve=SECP256k1)
vk = sk.verifying_key

# Uncompressed & compressed public keys
x = vk.pubkey.point.x(); y = vk.pubkey.point.y()
x_bytes = x.to_bytes(32, 'big'); y_bytes = y.to_bytes(32, 'big')
uncompressed_pub = b'\x04' + x_bytes + y_bytes
prefix = b'\x02' if (y % 2 == 0) else b'\x03'
compressed_pub = prefix + x_bytes

print('Priv (hex):', priv_key_bytes.hex())
print('Pub (uncompressed):', uncompressed_pub.hex())
print('Pub (compressed):', compressed_pub.hex())

# WIF using Base58Check from section 1
def wif_from_privkey(priv: bytes, compressed=True, mainnet=True):
    version = b'\x80' if mainnet else b'\xEF'
    payload = priv + (b'\x01' if compressed else b'')
    return base58check_encode(version, payload)

wif = wif_from_privkey(priv_key_bytes, compressed=True, mainnet=True)
print('WIF:', wif)

# ECDSA sign/verify demo (deterministic k)
msg = b'bitcoin is built on signatures'
msg_hash = hashlib.sha256(msg).digest()
sig = sk.sign_deterministic(msg_hash)
print('Signature (DER):', sig.hex())
print('Verify OK?:', vk.verify(sig, msg_hash))

Priv (hex): 874574494694ebe69557b88861e025cf58ca98b15aea64d57c69946b0bba99ce
Pub (uncompressed): 0468f26bc3b0f1a2d2a18d3a45a1bf10ab86145fd6c9893e9af3f8a9938f0fa26614eb8f76afeda81dea8acbc7e1700c93d735548ac9d766702b25a0c52e58bbbb
Pub (compressed): 0368f26bc3b0f1a2d2a18d3a45a1bf10ab86145fd6c9893e9af3f8a9938f0fa266
WIF: L1kfGb822z3bekis1jg5nQXnHpZeXScQk8tPfy6PErkjCPenjLeG
Signature (DER): 21153e38500c46fb2f05214b1f909a9744cbcb182a3d2bcd13dd3860401859fcc8db080e3fb20aeb5078bd385b441ac6ecec7d64216df98c8374ea05c338c2cb
Verify OK?: True


## 3) Bitcoin P2PKH (legacy `1…`) address <a id='p2pkh'></a>
`address = Base58Check(version=0x00, payload=HASH160(pubkey))`.

In [18]:
import hashlib

def hash160(b: bytes) -> bytes:
    return hashlib.new('ripemd160', hashlib.sha256(b).digest()).digest()

def p2pkh_address_from_pubkey(pubkey_bytes: bytes, mainnet=True) -> str:
    vh160 = (b'\x00' if mainnet else b'\x6F') + hash160(pubkey_bytes)
    return base58check_encode(vh160[:1], vh160[1:])

addr_legacy = p2pkh_address_from_pubkey(compressed_pub, mainnet=True)
print('P2PKH (legacy) address:', addr_legacy)

P2PKH (legacy) address: 1JeibuZCh2KFM6VgYyEppAeVhEbjo66HFc


## 4) Bitcoin P2SH (`3…`) from a redeem script <a id='p2sh'></a>
`address = Base58Check(version=0x05, payload=HASH160(redeem_script))`. Useful for multisig & custom scripts.

In [19]:
from binascii import unhexlify
import os

def push(data: bytes) -> bytes:
    return bytes([len(data)]) + data

# Example 2-of-3 multisig redeem script (illustrative; not production encoding)
pubkeys = [compressed_pub, os.urandom(33), os.urandom(33)]
redeem_script = b'\x52' + b''.join(push(pk) for pk in pubkeys) + b'\x53' + b'\xae'
sh = hash160(redeem_script)
version = b'\x05'
p2sh_addr = base58check_encode(version, sh)
print('P2SH address:', p2sh_addr)

P2SH address: 3HXG8XQ61o9dh8D8xP9q1fvy13aHAVDVYn


## 5) Bech32 (SegWit v0) P2WPKH `bc1…` <a id='bech32'></a>
Encodes `OP_0 <20-byte HASH160(pubkey)>` using Bech32 (BIP‑0173).

In [20]:
CHARSET = 'qpzry9x8gf2tvdw0s3jn54khce6mua7l'

def bech32_polymod(values):
    GEN = [0x3b6a57b2,0x26508e6d,0x1ea119fa,0x3d4233dd,0x2a1462b3]
    chk = 1
    for v in values:
        b = (chk >> 25) & 0xff
        chk = ((chk & 0x1ffffff) << 5) ^ v
        for i in range(5):
            chk ^= GEN[i] if ((b >> i) & 1) else 0
    return chk

def bech32_hrp_expand(hrp):
    return [ord(x) >> 5 for x in hrp] + [0] + [ord(x) & 31 for x in hrp]

def bech32_create_checksum(hrp, data):
    values = bech32_hrp_expand(hrp) + data
    polymod = bech32_polymod(values + [0,0,0,0,0,0]) ^ 1
    return [(polymod >> 5*(5-i)) & 31 for i in range(6)]

def bech32_encode(hrp, data):
    combined = data + bech32_create_checksum(hrp, data)
    return hrp + '1' + ''.join(CHARSET[d] for d in combined)

def convertbits(data, frombits, tobits, pad=True):
    acc = 0; bits = 0; ret = []
    maxv = (1 << tobits) - 1
    for value in data:
        if value < 0 or (value >> frombits): return None
        acc = (acc << frombits) | value
        bits += frombits
        while bits >= tobits:
            bits -= tobits
            ret.append((acc >> bits) & maxv)
    if pad and bits:
        ret.append((acc << (tobits - bits)) & maxv)
    elif bits >= frombits or ((acc << (tobits - bits)) & maxv):
        return None
    return ret

def hash160(b: bytes) -> bytes:
    import hashlib
    return hashlib.new('ripemd160', hashlib.sha256(b).digest()).digest()

def p2wpkh_address(pubkey_bytes: bytes, hrp: str = 'bc') -> str:
    h160 = hash160(pubkey_bytes)
    data = [0] + convertbits(h160, 8, 5)
    return bech32_encode(hrp, data)

bech32_addr = p2wpkh_address(compressed_pub, 'bc')
print('Bech32 P2WPKH:', bech32_addr)

Bech32 P2WPKH: bc1qcxwyrgtxtr7k5kzchpandjl0l4fzzlvwp3kakr


## 6) Bech32m (v1+, e.g., Taproot) <a id='bech32m'></a>
Bech32m changes the checksum constant (BIP‑350) and is required for witness versions ≥ 1.

In [21]:
def bech32m_create_checksum(hrp, data):
    values = bech32_hrp_expand(hrp) + data
    polymod = bech32_polymod(values + [0,0,0,0,0,0]) ^ 0x2bc830a3
    return [(polymod >> 5*(5-i)) & 31 for i in range(6)]

def bech32m_encode(hrp, data):
    combined = data + bech32m_create_checksum(hrp, data)
    return hrp + '1' + ''.join(CHARSET[d] for d in combined)

import os
taproot_program = os.urandom(32)  # illustrative 32‑byte witness program
data = [1] + convertbits(taproot_program, 8, 5)
print('Bech32m (v1+):', bech32m_encode('bc', data))

Bech32m (v1+): bc1p5e03hxeyxqcf2pu9yaj4yf9nrcs76qccv0hsrpdxzkzqdpwrdz0spy7n4m


## 7) Ethereum: Keccak‑256, EOA address derivation, and EIP‑55 <a id='ethereum'></a>
Ethereum uses Keccak‑256 (not FIPS SHA‑3) over the 64‑byte uncompressed pubkey; address is the last 20 bytes; EIP‑55 adds mixed‑case checksum.

In [22]:
from eth_keys import keys
from eth_utils import keccak, to_checksum_address
import os

eth_priv = os.urandom(32)
eth_sk = keys.PrivateKey(eth_priv)
eth_vk = eth_sk.public_key
uncompressed_64 = eth_vk.to_bytes()  # 64 bytes X||Y
eth_addr_bytes = keccak(uncompressed_64)[-20:]
eth_addr_hex = '0x' + eth_addr_bytes.hex()
print('Raw address (no checksum):', eth_addr_hex)
eth_addr_eip55 = to_checksum_address(eth_addr_hex)
print('EIP-55 address:', eth_addr_eip55)

message = b'hello ethereum'
msg_hash = keccak(message)
signature = eth_sk.sign_msg_hash(msg_hash)
print('Signature (r||s||v):', signature.to_bytes().hex())
print('Verify OK?:', eth_vk.verify_msg_hash(msg_hash, signature))

Raw address (no checksum): 0xdbffdbd1b006a1d2833b54fa9d15ff89400ccdf0
EIP-55 address: 0xDBFfDbD1B006A1d2833B54fA9D15ff89400CcdF0
Signature (r||s||v): 3ef397d9dd14d97ad51f525df65375ba199dfbd017206ebebc67947d9a16a34679844be67dfffe802a629260f6ccdb38e6a158e6332f634bfc5d2181e146f5a200
Verify OK?: True


## 8) EIP‑55 checksum (pure‑Python helper) <a id='eip55'></a>

In [23]:
from eth_utils import keccak

def to_eip55(addr_hex_no_0x: str) -> str:
    addr = addr_hex_no_0x.lower()
    hash_hex = keccak(addr.encode()).hex()
    checksummed = ''.join(
        ch.upper() if ch in 'abcdef' and int(hash_hex[i], 16) >= 8 else ch
        for i, ch in enumerate(addr)
    )
    return '0x' + checksummed

# Example:
print(to_eip55('a0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'))  # USDC mainnet contract

0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48


## 9) SHA‑256 vs Keccak‑256 quick compare <a id='hashcompare'></a>
Demonstrate the difference between `hashlib.sha3_256` (FIPS SHA‑3) and Ethereum's Keccak‑256.

In [24]:
import hashlib
from eth_utils import keccak
data = b'abc'
print('SHA-256:', hashlib.sha256(data).hexdigest())
print('SHA3-256 (FIPS):', hashlib.sha3_256(data).hexdigest())
print('Keccak-256:', keccak(data).hex())

SHA-256: ba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad
SHA3-256 (FIPS): 3a985da74fe225b2045c172d6bd390bd855f086e3e9d525b46bfe24511431532
Keccak-256: 4e03657aea45a94fc7d47ba826c8d667c0d1e6e33a64a036ec44f58fa12d6c45


## 10) Checksum demo: error detection <a id='checksumdemo'></a>
Flip a character in a Base58Check address and show that decoding fails.

In [25]:
# Requires addr_legacy from earlier cells
bad = addr_legacy[:5] + ('1' if addr_legacy[5] != '1' else '2') + addr_legacy[6:]
try:
    base58check_decode(bad)
    print('Unexpectedly valid!')
except ValueError as e:
    print('Checksum caught the error:', e)

Checksum caught the error: Bad Base58Check checksum
